## 异常商品识别

In [1]:
import pandas as pd
from keywords import *
import re

In [ ]:
#load data
goods = pd.read_csv('./data/goods_17w.csv')
goods = goods[['uid','shopNick','shopName','shopId','ordNum','goodsName','originalPrice','goodsNum']].drop_duplicates()
goods = goods[goods.goodsName!='保险服务']
goods.shape,goods.head()

/home/sf/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data = goods['goodsName'].drop_duplicates().dropna()
data #4289743

### 1.规则筛选

In [ ]:
def parse(title):
    if type(title) == type(u""):
        title = title.encode("utf-8").replace(" ", "")
    return_1 = list(set(re.findall(str.upper(keywords_1), str.upper(title))))
    fscore_1 = len(return_1)
    return_11 = list(set(re.findall(str.upper(keywords_11), str.upper(title))))
    fscore_11 = len(return_11)
    return_2 = list(set(re.findall(str.upper(keywords_2), str.upper(title))))
    fscore_2 = len(return_2)
    return_22 = list(set(re.findall(str.upper(keywords_22), str.upper(title))))
    fscore_22 = len(return_22)
    return_3 = list(set(re.findall(str.upper(keywords_3), str.upper(title))))
    fscore_3 = len(return_3)
    return_4 = list(set(re.findall(str.upper(keywords_4), str.upper(title))))
    fscore_4 = len(return_4)
    keywords_d = '|'.join(list(set(re.findall(str.upper(normal_words), str.upper(title)))))
    keywords = '|'.join(set(return_1 + return_11 + return_2 + return_22 + return_3 + return_4))
    #filter
    return_list = [fscore_1, fscore_11, fscore_2, fscore_22, fscore_3, fscore_4, keywords,keywords_d]
    return return_list

In [ ]:
data =data.astype(str)
score = data.apply(parse)
good_score=pd.DataFrame()
good_score['goodsName'] = data
good_score['score_1'] = score.apply(lambda x:x[0])
good_score['score_11'] = score.apply(lambda x:x[1])
good_score['score_2'] = score.apply(lambda x:x[2])
good_score['score_22'] = score.apply(lambda x:x[3])
good_score['score_3'] = score.apply(lambda x:x[4])
good_score['score_4'] = score.apply(lambda x:x[5])
good_score['keys_ab'] = score.apply(lambda x:x[6])
good_score['keys_norl'] = score.apply(lambda x:x[7])
del score,data
good_score.head()

In [ ]:
#abnormal category 1,2,4
ab_goods = pd.read_csv('./tmp/tmp1_check.csv',sep='\001')
ab_goods = ab_goods.drop_duplicates(subset=['goodsName'])
ab_goods['label']=1
#abnormal category 3
ab_goods_v = pd.read_csv('./tmp/tmp3_check.csv',sep='\001')
ab_goods_v['label'] =3
ab_goods_v
#merge :6877
ab_goods = ab_goods.append(ab_goods_v,ignore_index=True)
ab_goods =ab_goods.drop_duplicates(subset=['goodsName'])
ab_goods

In [ ]:
# #filter and hand check
# good_score[good_score.score_1>0].to_csv('./tmp/tmp1.csv',index=None,sep='\001')

# good_score[(good_score.score_11>1) & (good_score.score_1==0)].to_csv('./tmp/tmp11.csv',index=None,sep='\001')

# good_score[good_score.score_2>1].to_csv('./tmp/tmp2.csv',index=None,sep='\001')

# good_score[good_score.score_22>1].to_csv('./tmp/tmp22.csv',index=None,sep='\001')

# good_score[(good_score.score_3>1)& (good_score.score_1==0) & (good_score.score_2<2) &(good_score.score_11<2)&(good_score.score_22<2)].to_csv('./tmp/tmp3.csv',index=None,sep='\001')

# good_score[good_score.score_4>1].to_csv('./tmp/tmp4.csv',index=None,sep='\001')

### 统计异常

In [2]:
#异常商品购物记录 23945
ab_record = ab_goods.merge(goods,on='goodsName',how='left')
ab_record

NameError: name 'ab_goods' is not defined

In [ ]:
#异常用户
ab_usr = pd.DataFrame()
ab_record['goodsNum'] = ab_record['goodsNum'].fillna(1).astype(int)
ab_usr['buy_ab_nums'] = ab_record.groupby(by=['uid'])['goodsNum'].sum()
ab_usr['buy_ab_counts'] = ab_record.groupby(by=['uid'])['goodsName'].count()
ab_usr['buy_abShop_counts'] = ab_record.groupby(by=['uid'])['shopNick'].apply(lambda a:len(set(a)))
ab_usr['buy_ab_category'] = ab_record.groupby(by=['uid'])['label'].apply(lambda a:set(a))
ab_usr['avg_ab_num']=1.0*ab_usr['buy_ab_nums']/ab_usr['buy_ab_counts']
ab_usr = ab_usr.sort_values(by='buy_ab_nums',ascending=False)
ab_usr.to_csv('./data/abnormal_usr.csv',sep='\001',index=None)
ab_usr

In [ ]:
#异常店铺 5108
ab_shop = pd.DataFrame()
ab_shop['ab_good_num']=ab_record.groupby(by=['shopNick'])['goodsName'].apply(lambda a:len(set(a)))
ab_shop['ab_good_sale_count']=ab_record.groupby(by=['shopNick'])['goodsName'].count()
ab_shop['ab_good_sale_nums']=ab_record.groupby(by=['shopNick'])['goodsNum'].sum()
ab_shop['buy_usr_nums']=ab_record.groupby(by=['shopNick'])['uid'].apply(lambda a:len(set(a)))
ab_shop =ab_shop.sort_values(by='ab_good_num',ascending=False)
ab_shop.to_csv('./data/abnormal_shop.csv',sep='\001',index=None)
ab_shop

In [ ]:
#异常商品
ab_goods_s = pd.DataFrame()
ab_goods_s['sale_count'] = ab_record.groupby(by=['goodsName'])['uid'].count()
ab_goods_s['sale_nums'] = ab_record.groupby(by=['goodsName'])['goodsNum'].sum()
ab_goods_s['buy_usr_count'] = ab_record.groupby(by=['goodsName'])['uid'].apply(lambda a:len(set(a)))
ab_goods_s= ab_goods_s.sort_values(by='sale_count',ascending=False)
ab_goods = ab_goods_s.reset_index().merge(ab_goods,on='goodsName',how='left')
ab_goods.to_csv('./data/abnormal_goods.csv',sep='\001',index=None) 
ab_goods

## 异常商品与用户分析

In [4]:
ab_goods = pd.read_csv('./data/abnormal_goods.csv',sep='\001',index_col=0) 
ab_goods

,goodsName,sale_count,sale_nums,buy_usr_count,score_1,score_11,score_2,score_22,score_3,score_4,keys_ab,keys_norl,label
0,正规上市公司单位企业邮箱 全新账号支持绑定\闪银\51人品贷认证,1045,1048,930,1.0,0.0,0.0,3.0,2.0,0.0,账号|企业邮箱|闪银|51人品,NaN,1.0
1,正规世界500强企业单位邮箱 借趣花闪银提额认证企业单位邮箱,484,486,442,1.0,1.0,1.0,3.0,1.0,0.0,提额|闪银提额|邮箱|闪银|借趣花,NaN,1.0
2,拍拍贷 拍拍贷满标 拍拍贷快速满标 诚信低价 包满包到账不满退款,372,12860,298,2.0,0.0,0.0,0.0,0.0,0.0,满标|拍拍贷,NaN,1.0
3,GIF快手定制视频粉丝低价一万刷粉丝低价关注活粉涨粉评论上热门,290,1944,173,0.0,0.0,0.0,0.0,2.0,0.0,刷|评论,视频,3.0
4,正规上市集团 单位企业邮箱全新帐号 支持闪银 51人品 闪电等认证,259,259,232,1.0,0.0,0.0,4.0,1.0,0.0,闪电|企业邮箱|闪银|51人品,NaN,1.0
5,正规上市公司企业单位邮箱 支持绑定\闪银\51人品\闪电等认证专用,256,257,231,1.0,0.0,0.0,4.0,1.0,0.0,闪电|邮箱|闪银|51人品,NaN,1.0
6,拍拍贷 拍拍贷满标 你我贷 快速满标 嘉卡贷 曹操 还还 诚信低价,249,12817,217,4.0,0.0,0.0,0.0,0.0,0.0,你我贷|满标|拍拍贷|嘉卡贷,NaN,1.0
7,世界500强公司单位企业邮箱全新帐号 支持闪银 借趣花 51 人品贷,242,242,221,1.0,0.0,0.0,3.0,1.0,0.0,企业邮箱|闪银|借趣花,NaN,1.0
8,拍拍贷 拍拍贷满标 快速满标,224,10903,188,2.0,0.0,0.0,0.0,0.0,0.0,满标|拍拍贷,NaN,1.0
9,GIF快手定制设计小视频制作不刷粉丝双击喜欢播放量评论上热门,194,2102,110,0.0,0.0,0.0,0.0,2.0,0.0,刷|评论,设计|视频,3.0


In [ ]:
#所有商品特征统计
goods_s = pd.Dataframe()
goods_s['sale_count'] = goods.groupby(by=['goodsName'])['uid'].count()
goods_s['sale_nums'] = goods.groupby(by=['goodsName'])['goodsNum'].sum()
goods_s['buy_usr_count'] = goods.groupby(by=['goodsName'])['uid'].apply(lambda a:len(set(a)))

In [ ]:
goods_s = goods_s.merge(ab_goods[['goodsName','label']],on='goodsName',how='left')
goods_s['label'].fillna(0,inplace=True)
goods_s

In [ ]:
#所有店铺特征统计
shop = pd.DataFrame()
shop['good_num']=goods.groupby(by=['shopNick'])['goodsName'].apply(lambda a:len(set(a)))
shop['good_sale_count']=goods.groupby(by=['shopNick'])['goodsName'].count()
shop['good_sale_nums']=goods.groupby(by=['shopNick'])['goodsNum'].sum()
shop['avg_sale_nums']=goods.groupby(by=['shopNick'])['goodsNum'].mean()
shop['avg_price']=goods.groupby(by=['shopNick'])['originalPrice'].mean()
#load
tmp = pd.read_csv('./data/shop_feature_17w.csv')
shop = shop.merge(tmp,on='shopNick',how='left')
shop =shop.sort_values(by='good_num',ascending=False)
shop.to_csv('./data/shop_feature_all_17w.csv',sep='\001')
shop

### 2. 模型筛选
#### 1.naive bayes

In [1]:
import keras


ImportError: No module named keras